#### Conectando Python com API do Google Sheets

In [ ]:
import pandas as pd
import gspread
import json
import os
import unidecode
import warnings

warnings.filterwarnings("ignore")
from dotenv import load_dotenv

def connect_sheets(sheet_id, sheet_name):
    """
    Realiza a conexão com a API do google sheets,
    Verifica se existem dados nos parâmetros informados
    """
    # Carrega as variáveis de ambiente
    load_dotenv()
    key_json = os.getenv('KEY_JSON')

    # Verifica se a chave JSON foi encontrada
    if not key_json:
        raise ValueError("Chave JSON não encontrada.")

    # Conexão com o Google Sheets
    google_client = gspread.service_account_from_dict(json.loads(key_json))
    worksheet = google_client.open_by_key(sheet_id).worksheet(sheet_name)

    # Obtenção dos dados e conversão para DataFrame
    dados = worksheet.get_all_values()
    if not dados:
        raise ValueError("A planilha está vazia.")

    return pd.DataFrame(dados[1:], columns=dados[0])

df = connect_sheets('1tbJh_GQol0Ax5uB-AjwI26yW9piCIp0peuKV1N5nMeQ', 'db')

In [ ]:
df.head()

#### Limpeza e Preparação dos dados

In [ ]:
# Padronizando o nome das colunas
df.rename(columns=str.lower, inplace=True)
df.columns = df.columns.str.replace(' ', '_')

# Removendo os acentos das colunas
df.columns = [unidecode.unidecode(col) for col in df.columns]

df.columns

In [ ]:
# Converter colunas de data/hora para o tipo datetime
df['chegada_cd'] = pd.to_datetime(df['chegada_cd'])
df['inicio_do_carregamento'] = pd.to_datetime(df['inicio_do_carregamento'])
df['fim_do_carregamento'] = pd.to_datetime(df['fim_do_carregamento'])
df['data'] = pd.to_datetime(df['data'], format='%d/%m/%Y')

# Lista de colunas que precisam ser convertidas para timedelta
timedelta_columns = [
    'tempo_de_espera', 
    'tempo_de_expedicao', 
    'tempo_total', 
    'hora_chegada_cd', 
    'hora_inicio_do_carregamento', 
    'hora_fim_do_carregamento'
]

# Aplicar a conversão para timedelta usando apply
df[timedelta_columns] = df[timedelta_columns].apply(pd.to_timedelta)

# Ajustar "quantidade_remessas" como tipo numérico (inteiro)
df['quantidade_remessas'] = pd.to_numeric(df['quantidade_remessas'], errors='coerce')

# Convertendo o tipo de dado para categorico
df[['unidade', 'status_meta']] = df[['unidade', 'status_meta']].astype('category')

In [ ]:
df.info()

In [ ]:
df.isna().sum()

#### Transformando os dados com `groupby`

In [ ]:
 # Tempo Médio de Expedição por Data
tempo_medio_dia = df.groupby(['data', 'unidade']).agg(
                    media_tempo_espera = pd.NamedAgg('tempo_de_espera', 'mean'),
                    media_tempo_expedicao = pd.NamedAgg('tempo_de_expedicao', 'mean'),
                    media_tempo_total = pd.NamedAgg('tempo_total', 'mean'),
                    total_remessas = pd.NamedAgg('quantidade_remessas', 'sum'),
                    total_motoristas = pd.NamedAgg('nome', 'count')).reset_index()

def format_timedelta(td):
    # Verifica se o valor é NaN
    if pd.isnull(td):
        return "00:00:00"
    
    # Caso contrário, realiza a conversão normalmente
    total_seconds = int(td.total_seconds())
    hours, remainder = divmod(total_seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    return f"{hours:02}:{minutes:02}:{seconds:02}"

# Aplicar a formatação das médias
tempo_medio_dia[['media_tempo_espera',
                 'media_tempo_expedicao',
                 'media_tempo_total']] = tempo_medio_dia[['media_tempo_espera',
                                                         'media_tempo_expedicao',
                                                         'media_tempo_total']].applymap(format_timedelta)

tempo_medio_dia = tempo_medio_dia[tempo_medio_dia['total_remessas'] > 0]

In [ ]:
tempo_medio_dia.to_clipboard(index=False)